In [39]:
import os
import random
import numpy as np
# Employ PIL for effective image processing capabilities
from PIL import Image
# Integrate tqdm to include progress bars for enhanced visibility in loops
from tqdm import tqdm
# Utilize Matplotlib and seaborn for comprehensive plotting capabilities
import matplotlib.pyplot as plt
# PyTorch for constructing and training neural networks
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
# Leverage torchvision for seamless management of image data and utilization of pretrained models
from torchvision import transforms
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from torchvision.models import resnet50, resnet152, ResNet50_Weights, ResNet152_Weights

## Initialization and Data Loading

In [40]:
# Function to set up the computation device
def initialize_device():
    computation_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Computation Device: {computation_device}")
    return computation_device

# Function to load dataset
def load_dataset(data_file, label_file):
    data = np.load(data_file)
    label_set = np.load(label_file)
    image_count = data.shape[1]
    reshaped_data = data.reshape(image_count, 3, 300, 300).transpose(3, 0, 1, 2)
    # return reshaped_data, label_set
    return data, label_set

# Initialize device
device = initialize_device()

# Load data and labels
test_data_file =  'data-1.npy'
test_label_file = 'labels-1.npy'
test_data, test_labels = load_dataset(test_data_file, test_label_file)


Computation Device: cuda


## Transformations and Dataset

In [41]:
# Function to define transformation pipeline
def create_transformations():
    image_transformations = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return image_transformations

# Custom dataset class
class ImageDataset(Dataset):
    def __init__(self, images, labels, transformations=None):
        self.images = images
        self.labels = labels
        self.transformations = transformations

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        image_data = self.images[:, index].reshape(300, 300, 3)
        pil_image = Image.fromarray(image_data)
        if self.transformations:
            transformed_image = self.transformations(pil_image)
        return transformed_image, self.labels[index]

# Create transformations
data_transformations = create_transformations()


## Load test data

In [42]:
# Load the data and labels from the .npy files
data_test = np.load(test_data_filepath)
labels_test = np.load(test_labels_filepath)
# Create a custom dataset using the loaded data and labels along with the predefined transformations
dataset = ImageDataset(data_test, labels_test, data_transformations)
load_test = DataLoader(dataset, batch_size=128)

In [43]:
# Function to initialize and load the trained model
def initialize_and_load_model(file_path, class_count):
    neural_network_model = resnet152(pretrained=False, num_classes=class_count)
    neural_network_model.load_state_dict(torch.load(file_path))
    return neural_network_model

# Function to perform the evaluation on the test dataset
def perform_test_evaluation(test_data_loader, evaluation_model, evaluation_criterion, device):
    accumulated_test_loss = 0
    total_correct_predictions = 0
    total_samples_evaluated = 0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for inputs, labels in test_data_loader:
            inputs, labels = inputs.to(device, dtype=torch.float), labels.to(device, dtype=torch.long)
            model_outputs = evaluation_model(inputs)
            accumulated_test_loss += evaluation_criterion(model_outputs, labels).item()
            _, predicted_labels = torch.max(model_outputs.data, 1)
            total_correct_predictions += (predicted_labels == labels).sum().item()
            total_samples_evaluated += labels.size(0)

    average_loss = accumulated_test_loss / len(test_data_loader.dataset)
    accuracy_percent = 100 * total_correct_predictions / total_samples_evaluated
    return average_loss, accuracy_percent

# Model and test configuration
model_file_path = 'model/resnet_model_final.pth' 
number_of_classes = 9  
loss_function = nn.CrossEntropyLoss()

# Load the trained model and perform evaluation
trained_model = initialize_and_load_model(model_file_path, number_of_classes).to(device)
avg_loss, test_acc = perform_test_evaluation(test_loader, trained_model, loss_function, device)

# Output the test results
print(f'Average Loss on Test Data: {avg_loss:.4f}, Overall Test Accuracy: {test_acc:.2f}%')


Average Loss on Test Data: 0.0025, Overall Test Accuracy: 97.04%
